In [1]:
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys
from rdkit.ML.Descriptors import MoleculeDescriptors

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

from mordred import Calculator, descriptors

# Import data file

In [4]:
data_version_1_path = "../data_for_modeling/filter_data/v1/HDAC2_data_after_filtering_raw_v1.xlsx"
data_version_2_path = "../data_for_modeling/filter_data/v2/HDAC2_data_after_filtering_raw_v2.xlsx"
dataset1 = pd.read_excel(data_version_1_path, sheet_name='Final data to train')
dataset2 = pd.read_excel(data_version_2_path, sheet_name='final_data')

In [5]:
dataset1.head()

,STT,SMILES,CID,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,0,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)C=CC(=O)N...,155525662,4.21400,Active,Inactive,1
1,1,CSC1=CC2=C(C=C1)SC3=CC=CC=C3N2CC4=CC=C(C=C4)C(...,164629157,0.68000,Active,Active,1
2,2,C1=CC=C2C(=C1)N(C3=C(S2=O)C=CC(=C3)C(F)(F)F)CC...,164627475,2.12000,Active,Inactive,1
3,3,CC(C)(C)OC(=O)NC1=CC=C(C=C1)C2=CC(=NO2)NC(=O)C...,164627446,0.25200,Active,Active,1
4,4,CCCC[C@@H](C1=NC=C(N1)C2=CC3=CC=CC=C3N=C2OC)NC...,164627330,2.00525,Active,Inactive,4


In [6]:
dataset2.head()

,STT,SMILES,CID,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,0,CN1CCC2=C(C1)C3=CC=CC=C3N2CC4=CC=C(C=C4)C(=O)NO,49850262,6.5749,Active,Inactive,5
1,1,C1=CC=C(C(=C1)N)NC(=O)CCCCNC(=O)/C(=N/O)/CC2=C...,162650384,9.7600,Active,Inactive,2
2,2,CC(C)(C)NC(=O)CN(CCCCCC(=O)NO)C(=O)C1=CC=CC=C1,137632943,7.4825,Active,Inactive,4
3,3,C1=CC(=C(C=C1C/C(=N\O)/C(=O)NCC(=O)NO)Br)O,162653508,9.2100,Active,Inactive,2
4,4,CC1=C(C2=C(C(=C1O)O)OC(=C2)C3=C(C(=C(C(=C3C=O)...,11210533,9.1000,Active,Inactive,1


In [7]:
concatenated_df = pd.concat([dataset1, dataset2])

In [8]:
concatenated_df.head()

,STT,SMILES,CID,AVG_IC50_uM,FIRST_LABEL,FINAL_LABEL,DUPLICATE_COUNTS
0,0,CC1=C(C2=CC=CC=C2N1)CCNCC3=CC=C(C=C3)C=CC(=O)N...,155525662,4.21400,Active,Inactive,1
1,1,CSC1=CC2=C(C=C1)SC3=CC=CC=C3N2CC4=CC=C(C=C4)C(...,164629157,0.68000,Active,Active,1
2,2,C1=CC=C2C(=C1)N(C3=C(S2=O)C=CC(=C3)C(F)(F)F)CC...,164627475,2.12000,Active,Inactive,1
3,3,CC(C)(C)OC(=O)NC1=CC=C(C=C1)C2=CC(=NO2)NC(=O)C...,164627446,0.25200,Active,Active,1
4,4,CCCC[C@@H](C1=NC=C(N1)C2=CC3=CC=CC=C3N=C2OC)NC...,164627330,2.00525,Active,Inactive,4


In [9]:
len(concatenated_df)

2820

# Cannocal Smile filter

In [10]:
def canonical_smiles(smiles):
    mols = [Chem.MolFromSmiles(smi) for smi in smiles]
    smiles = [Chem.MolToSmiles(mol) for mol in mols]
    return smiles

In [12]:
Canon_SMILES = canonical_smiles(concatenated_df.SMILES)
concatenated_df['SMILES'] = Canon_SMILES
duplicates_smiles = concatenated_df[concatenated_df['SMILES'].duplicated()]['SMILES'].values